In [1]:
import nidaqmx 
import atiiaftt as ati
import time
import numpy as np
from nidaqmx.stream_readers import AnalogMultiChannelReader
import matplotlib.pyplot as plt
import pandas as pd
from nidaqmx.constants import AcquisitionType
from pprint import pprint

start_task
close_task

read_raw_voltage
read_avg_voltage
read_raw_ft
read_avg_ft

set_bias(bias=None)

set_channel_number
set_sampling_freq
set_read_freq

get_calibration_file_path
get_tool_offset
get_channel_number
get_sampling_freq
get_read_freq
get_sample_number

In [18]:
class ftsensor:
    def __avg(lst):
        return sum(lst) / len(lst)

    def __init__(self, calibration_file_path="..\FT44764\FT44764.cal", tool_offset=[0,0,0,0,0,0]):
        self.calibration_file_path = calibration_file_path
        self.tool_offset = tool_offset
        self.channel_number = 6
        self.sampling_freq = 6400
        self.read_freq = 64
        self.sample_number = self.sampling_freq//self.read_freq
        self.raw_readings = np.zeros((self.channel_number, self.sample_number), dtype=np.float64)
        
        self.sensor = ati.FTSensor()
        self.calibrate(calibration_file_path=self.calibration_file_path)
        self.set_tool(self.tool_offset)

    def calibrate(self, calibration_file_path):
        self.sensor.createCalibration(calibration_file_path, 1)
        self.sensor.setForceUnits("N".encode("utf-8"))
        self.sensor.setTorqueUnits("N-m".encode("utf-8"))
        
    def set_tool(self, tool_offset):
        self.sensor.setToolTransform(tool_offset, "mm".encode("utf-8"), "deg".encode("utf-8"))

    def start_task(self):
        self.task = nidaqmx.Task()
        for i in range(self.channel_number):
            self.task.ai_channels.add_ai_voltage_chan(f"Dev1/ai{i}")
        self.task.timing.cfg_samp_clk_timing(self.sampling_freq, sample_mode = AcquisitionType.CONTINUOUS, samps_per_chan = self.sample_number)
        self.reader = AnalogMultiChannelReader(self.task.in_stream)
    
    def close_task(self):
        self.task.close()

    def read_raw_voltage(self):
        self.reader.read_many_sample(data=self.raw_readings, number_of_samples_per_channel=self.sample_number, timeout=1/self.read_freq)
        return self.raw_readings

In [19]:
sensor = ftsensor()
sensor.start_task()
print(sensor.read_raw_voltage())
sensor.close_task()

c:\users\andyb\appdata\local\programs\python\python310\lib\site-packages\nidaqmx\task.py:97: ResourceWarning: Task of name "_unnamedTask<3>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(


DaqError: Device cannot be accessed.  Possible causes:
Device is no longer present in the system.
Device is not powered.
Device is powered, but was temporarily without power.
Device and/or chassis driver support may have been removed.
Device is damaged.
Ensure the device is properly connected and turned on. Ensure the device and/or chassis is supported in the current version of the driver. Check the device's status in NI MAX. Turn the computer off and on again. If you suspect that the device is damaged, contact National Instruments at ni.com/support.
Device Specified: Dev1

Task Name: _unnamedTask<4>

Status Code: -201003

In [5]:
CHANNEL_NUM = 6
SAMPLING_FREQ = 6400
READ_FREQ = 64
SAMPLE_NUM = SAMPLING_FREQ//READ_FREQ
values = np.zeros((CHANNEL_NUM, SAMPLE_NUM), dtype=np.float64)

DURATION = 5

sensor = ati.FTSensor()

CALFILEPATH = "..\FT44764\FT44764.cal"
INDEX = 1
cal = sensor.createCalibration(CALFILEPATH, INDEX)

sensor.setForceUnits("N".encode("utf-8"))
sensor.setTorqueUnits("N-m".encode("utf-8"))

TOOL_OFFSET = [0,0,0,0,0,0]
sensor.setToolTransform(TOOL_OFFSET, "mm".encode("utf-8"), "deg".encode("utf-8"))

with nidaqmx.Task() as task:
    for i in range(CHANNEL_NUM):
        task.ai_channels.add_ai_voltage_chan(f"Dev1/ai{i}")

    task.timing.cfg_samp_clk_timing(SAMPLING_FREQ, sample_mode = AcquisitionType.CONTINUOUS, samps_per_chan = SAMPLE_NUM)
    reader = AnalogMultiChannelReader(task.in_stream)
    
    reader.read_many_sample(data=values, number_of_samples_per_channel=SAMPLE_NUM, timeout=1/READ_FREQ)

DaqError: Device cannot be accessed.  Possible causes:
Device is no longer present in the system.
Device is not powered.
Device is powered, but was temporarily without power.
Device and/or chassis driver support may have been removed.
Device is damaged.
Ensure the device is properly connected and turned on. Ensure the device and/or chassis is supported in the current version of the driver. Check the device's status in NI MAX. Turn the computer off and on again. If you suspect that the device is damaged, contact National Instruments at ni.com/support.
Device Specified: Dev1

Task Name: _unnamedTask<1>

Status Code: -201003

In [69]:
current_time = time.time()
readings_inside_loop = []
while time.time() < current_time+DURATION:
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai5")

        readings_inside_loop.append(sensor.convertToFt(task.read()))
readings_inside_loop = np.array(readings_inside_loop)

print("Shape size: ", readings_inside_loop.shape)
std_readings_inside_loop = readings_inside_loop.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_inside_loop)
avg_readings_inside_loop = readings_inside_loop.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_inside_loop)

Shape size:  (178, 6)
Standard Deviation of Force X (N):  0.26224823465651237
Average of Force X (N):  1.3109888725736167


In [70]:
current_time = time.time()
readings_outside_loop = []
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    while time.time() < current_time+DURATION:
        readings_outside_loop.append(sensor.convertToFt(task.read()))
readings_outside_loop = np.array(readings_outside_loop)


print("Shape size: ", readings_outside_loop.shape)
std_readings_outside_loop = readings_outside_loop.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_outside_loop)
avg_readings_outside_loop = readings_outside_loop.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_outside_loop)

Shape size:  (202, 6)
Standard Deviation of Force X (N):  0.16941986935109848
Average of Force X (N):  1.320638337347767


In [71]:
readings_multi_read_one = []
with nidaqmx.Task() as read_task:
    read_task = nidaqmx.Task()
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    reader = AnalogMultiChannelReader(read_task.in_stream,)
    values_read = np.zeros(6, dtype=np.float64)



    current_time = time.time()
    while time.time() < current_time+DURATION:
        reader.read_one_sample(values_read)
        
        readings_multi_read_one.append(sensor.convertToFt(values_read.tolist()))

readings_multi_read_one = np.array(readings_multi_read_one)


print("Shape size: ", readings_multi_read_one.shape)
std_readings_multi_read_one = readings_multi_read_one.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_multi_read_one)
avg_readings_multi_read_one = readings_multi_read_one.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_multi_read_one)

Shape size:  (222, 6)
Standard Deviation of Force X (N):  0.18509847560472495
Average of Force X (N):  1.2915892128471855


In [72]:
readings_multi_read_many = []
with nidaqmx.Task() as read_task:
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai5")

    sample_rate = 6400
    samples_to_acq = DURATION * sample_rate
    values_read = np.zeros((6, samples_to_acq), dtype=np.float64)
    cont_mode = AcquisitionType.CONTINUOUS
    read_task.timing.cfg_samp_clk_timing(sample_rate, sample_mode = cont_mode, samps_per_chan = samples_to_acq)               
    reader = AnalogMultiChannelReader(read_task.in_stream)

    current_time = time.time()
    reader.read_many_sample(data=values_read, number_of_samples_per_channel=samples_to_acq, timeout=11)
    test_time = time.time()
    print("time passed: ", test_time-current_time,"(s)")

readings_multi_read_many = np.array([np.array(channel) for channel in values_read]).transpose()
readings_multi_read_many = np.array([sensor.convertToFt(reading.tolist()) for reading in readings_multi_read_many])

std_readings_multi_read_many = readings_multi_read_many.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_multi_read_many)
avg_readings_multi_read_many = readings_multi_read_many.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_multi_read_many)

time passed:  5.024990797042847 (s)
Standard Deviation of Force X (N):  0.2332736744298239
Average of Force X (N):  1.3055461626704783


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\nidaqmx\task.py:97: ResourceWarning: Task of name "_unnamedTask<258>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(


In [115]:
sample_rate = 6400
samples_to_acq = DURATION * sample_rate
values_read = np.zeros((6, samples_to_acq), dtype=np.float64)

with nidaqmx.Task() as read_task:
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    
    read_task.timing.cfg_samp_clk_timing(sample_rate, sample_mode = AcquisitionType.CONTINUOUS, samps_per_chan = samples_to_acq)               
    
    reader = AnalogMultiChannelReader(read_task.in_stream)

    current_time = time.time()
    reader.read_many_sample(data=values_read, number_of_samples_per_channel=samples_to_acq, timeout=DURATION)
    test_time = time.time()
    print("time passed: ", test_time-current_time,"(s)")

final_readings = [[__avg(sample) for sample in channel] for channel in np.reshape(values_read, (6, 32000//100,100))]

final_ft = np.array([sensor.convertToFt([final_readings[i][j] for i in range(len(final_readings))]) for j in range(len(final_readings[0]))])

std_final_ft = final_ft.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_final_ft)
avg_final_ft = final_ft.transpose()[0].mean()
print("Average of Force X (N): ", avg_final_ft)

time passed:  5.024467706680298 (s)
Standard Deviation of Force X (N):  0.0009793974322685006
Average of Force X (N):  1.289738005027175


In [114]:
sample_rate = 6400
samples_read = 100

with nidaqmx.Task() as read_task:
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    
    read_task.timing.cfg_samp_clk_timing(sample_rate, sample_mode = AcquisitionType.CONTINUOUS, samps_per_chan = samples_read)               
    
    read_task.start()
    reading = task.read(number_of_samples_per_channel=samples_read)
    read_task.stop()
    print(readings)

final_readings = [[__avg(sample) for sample in channel] for channel in np.reshape(values_read, (6, 32000//100,100))]

final_ft = np.array([sensor.convertToFt([final_readings[i][j] for i in range(len(final_readings))]) for j in range(len(final_readings[0]))])

std_final_ft = final_ft.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_final_ft)
avg_final_ft = final_ft.transpose()[0].mean()
print("Average of Force X (N): ", avg_final_ft)

[-0.12558438 -0.12561725 -0.12558602 -0.12558438 -0.1255778  -0.12554987
 -0.12561232 -0.12560903 -0.1256041  -0.12557287 -0.12554987 -0.12558767
 -0.12558602 -0.12557123 -0.12559424 -0.1255663  -0.12558767 -0.12555808
 -0.12554822 -0.12558767 -0.12562382 -0.1256041  -0.12561232 -0.12555315
 -0.12557123 -0.12555644 -0.12553015 -0.12555808 -0.12557616 -0.12562053
 -0.12557616 -0.12560574 -0.12555644 -0.12557452 -0.12550385 -0.12556301
 -0.12560081 -0.12561889 -0.12558109 -0.12557452 -0.1255548  -0.12555151
 -0.12556794 -0.1255778  -0.12559753 -0.12558602 -0.12556794 -0.12559753
 -0.12557945 -0.12554329 -0.12553015 -0.12554658 -0.12558602 -0.12557123
 -0.1256156  -0.12558438 -0.1255663  -0.12554329 -0.12551864 -0.12556959
 -0.1255926  -0.1256041  -0.12559424 -0.12563697 -0.12558767 -0.12556959
 -0.12555808 -0.1255548  -0.12554987 -0.12560246 -0.12557123 -0.12556466
 -0.12559588 -0.1255663  -0.12553508 -0.12556794 -0.12558438 -0.12555808
 -0.12558931 -0.12558602 -0.1255548  -0.12556301 -0

In [113]:
print((test_reading[0]))

[-0.12558437850724577, -0.1256172469540764, -0.1255860219295842, -0.12558437850724913, -0.12557780481788647, -0.12554986663806772, -0.12561231668705214, -0.12560902984237296, -0.1256040995753449, -0.12557287455085228, -0.12554986663807818, -0.125587665351924, -0.12558602192958132, -0.1255712311285205, -0.12559423904129996, -0.12556630086149156, -0.1255876653519301, -0.1255580837497763, -0.12554822321573306, -0.1255876653519335, -0.12562382064344946, -0.12560409957534358, -0.12561231668705694, -0.1255531534827602, -0.12557123112851853, -0.12555644032744034, -0.12553014556997089, -0.12555808374978297, -0.12557616139553562, -0.12562053379876217, -0.12557616139557304, -0.12560574299768004, -0.1255564403274351, -0.12557451797319164, -0.12550385081252272, -0.12556301401680622, -0.12560081273066248, -0.12561889037642618, -0.12558109166256665, -0.1255745179731982, -0.12555479690510044, -0.1255515100604161, -0.12556794428382945, -0.12557780481788078, -0.1255975258859767, -0.12558602192957846, -

In [83]:
FOLDER_DIR = "..\Joey\Record_Data"

demo_df = pd.read_csv(FOLDER_DIR+"\Demo\Sensor_2.csv")

demo_df.drop(demo_df.columns[6:], axis=1, inplace=True)

print("Shape size: ", demo_df.shape)
demo_df["Force X (N)"].astype(int)
# print("Standard Deviation of Force X (N): ", demo_df["Force X (N)"].std())
# print("Average of Force X (N): ", demo_df["Force X (N)"].mean())

Shape size:  (813, 6)


ValueError: invalid literal for int() with base 10: '1.19507384300232'